In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_election_results(url, table_class):
    # Make a GET request to the URL
    response = requests.get(url)
    data = []

    try:
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the table with the results
            table = soup.find('table', {'class': table_class})

            # Extract table headers
            headers = [header.get_text(strip=True) for header in table.find('thead').find_all('th')]
            headers.append('Link')  # Add a new header for the links

            # Extract table rows
            rows = table.find('tbody').find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                row_data = {headers[i]: cols[i].get_text(strip=True) for i in range(len(headers) - 1)}

 # Check for anchor tags and add the link to the row data
                for col in cols:
                    link_tag = col.find('a')
                    if link_tag:
                        row_data['Link'] = link_tag['href']
                        break  # Assuming one link per row, break after finding the first link

                else:
                    row_data['Link'] = None  # If no link is found, set to None

                data.append(row_data)

            # Create DataFrame from the data
            df = pd.DataFrame(data)
            df['Link']= row_data['Link']='https://results.eci.gov.in/PcResultGenJune2024/'+df['Link']
            return df
        else:
            print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(e)
        return None